In [1]:
import random
import pandas as pd
import numpy as np

In [29]:
def getSourceWords():
    with open('source_words.txt', 'r') as f:
        return f.read().splitlines()

words = getSourceWords()
print("Length : ",len(words))
print(words[4:10])

Length :  12972
['awake', 'blush', 'focal', 'evade', 'naval', 'serve']


In [2]:
class Word:
    def __init__(self, word):
        self.word = word

    def getFitness(self,target):
        if len(self.word) != len(target):
            return 0
        score = 0
        greenScore = 5
        yellowScore = 1
        for i in range(len(self.word)):
            if self.word[i] == target[i]:
                score += greenScore
            elif self.word[i] in target:
                score += yellowScore
        return score
    def __str__(self):
        return self.word

In [39]:
def makeGuess():
    return Word(random.choice(words))

In [73]:
L = []
for i in range(10):
    guess = makeGuess()
    f = guess.getFitness("guest")
    L.append((guess,f))

## sort by fitness
L.sort(key=lambda x: x[1], reverse=True)
print("Word : ",L[0][0])
print("Fitness : ",L[0][1])

Word :  gesse
Fitness :  13


In [53]:
def sortPopulation(population):
    
    return sorted(population, key=lambda x: x.getFitness("guest"), reverse=True)

fuels


In [72]:
class Wordle:
    def __init__(self, target):
        self.words = self.getSourceWords()
        self.target = target
        self.guesses = []
        self.bestGuess = None
        self.bestFitness = 0
        
    def getSourceWords(self):
        with open('source_words.txt', 'r') as f:
            return f.read().splitlines()

    def addGuess(self, guess):
        self.guesses.append(guess)
        fitness = guess.getFitness(self.target)
        if fitness > self.bestFitness:
            self.bestGuess = guess
            self.bestFitness = fitness

    def __str__(self):
        return self.bestGuess.word

    ## genetic algorithm

    def getInitialPopulation(self,size):
        population = random.sample(self.words,size)
        ret = []
        for i in range(len(population)):
            ret.append(Word(population[i]))
        return ret

    def sortPopulation(self, population):
        ret = []
        for i in range(len(population)):
            guess = population[i]
            ret.append((guess, guess.getFitness(self.target)))
        return sorted(ret, key=lambda x: x[1], reverse=True)

    def selection(self, sortedPopulation,eliteSize):
        selectionResults = []
        df = pd.DataFrame(np.array(sortedPopulation), columns=["Guess","Fitness"])
        df['cumulative_sum'] = df.Fitness.cumsum()
        df['cumulative_percentage'] = 100*df.cumulative_sum/df.Fitness.sum()
        # print(df)
        # print("----")
        
        ## ensure that the elits always get selected
        for i in range(0,eliteSize):
            selectionResults.append(sortedPopulation[i])

        # for i in range(len(selectionResults)):
        #     print("Selection : ",selectionResults[i][0], " , fitness : ",selectionResults[i][1])
        # print("----")
        
        ## randomly pick the others, make it the same size as initial population
        for i in range(0, len(sortedPopulation) - eliteSize):
            pick = random.uniform(0,100)
            for idx, row in df.iterrows():
                if row.cumulative_percentage >= pick:
                    selectionResults.append(sortedPopulation[idx])
                    break
        
        # for i in range(len(selectionResults)):
        #     print("Selection : ",selectionResults[i][0], " , fitness : ",selectionResults[i][1])

        return selectionResults

    def crossover(self, parent1, parent2):
        child1 = Word(parent1.word[:int(len(parent1.word)/2)] + parent2.word[int(len(parent2.word)/2):])
        child2 = Word(parent2.word[:int(len(parent2.word)/2)] + parent1.word[int(len(parent1.word)/2):])
        return child1, child2

    def breed(self,selected,eliteSize):
        length = len(selected) - eliteSize

        newPopulation = []
        for i in range(eliteSize):
            newPopulation.append(selected[i][0])

        for i in range(int(length/2)):
            parent1 = selected[i][0]
            parent2 = selected[(i+1)%length][0]
            child1,child2 = self.crossover(parent1, parent2)
            newPopulation.append(child1)
            newPopulation.append(child2)
        return newPopulation

    def mutate(self,population,mutationRate):
        for i in range(len(population)):
            if random.uniform(0,1) < mutationRate:
                ## change a random letter to a-z
                index = random.randint(0,len(population[i].word)-1)
                population[i].word = population[i].word[:index] + chr(random.randint(97,122)) + population[i].word[index+1:]
        return population

    def solve(self,initSize = 500,eliteSize = 0,generations=50,mutationRate=0.2): 
        population = self.getInitialPopulation(initSize)
        sortedPopulation = self.sortPopulation(population)
        print("Initial Best Guess : ",sortedPopulation[0][0], " , fitness : ",sortedPopulation[0][1])
 
        for g in range(generations):
            print("Generation : ",g)
            selected = self.selection(sortedPopulation,eliteSize)
            newPopulation = self.breed(selected,eliteSize)
            print("New population size : ",len(newPopulation))
            newPopulation = self.mutate(newPopulation,mutationRate)
            sortedPopulation = self.sortPopulation(newPopulation)
            if sortedPopulation[0][0].word == self.target:
                print("Found the target : ",sortedPopulation[0][0], "in generation : ",g+1)
                break
            print("Best Guess : ",sortedPopulation[0][0], " , fitness : ",sortedPopulation[0][1])


In [73]:
wordle = Wordle("guest")
solve = wordle.solve()

Initial Best Guess :  crews  , fitness :  6
Generation :  0
New population size :  10
Found the target :  guest in generation :  1


In [ ]:
def crossover(parent1, parent2):
    child1 = Word(parent1.word[:int(len(parent1.word)/2)] + parent2.word[int(len(parent2.word)/2):])
    child2 = Word(parent2.word[:int(len(parent2.word)/2)] + parent1.word[int(len(parent1.word)/2):])
    return child1, child2

In [23]:
c1,c2 = crossover(Word("guesti"), Word("abcdef"))
print(c1)
print(c2)

guedef
abcsti


In [53]:
def mutate(word,mutationRate=1):
    if random.uniform(0,1) < mutationRate:
        ## change a random letter to a-z
        index = random.randint(0,len(word)-1)
        word = word[:index] + chr(random.randint(97,122)) + word[index+1:]
    print(word)

mutate("hello")

mello
